# 1. Bibiliothèques et chargement des données

In [3]:
%%capture
!pip install datasets==1.4.1
!pip install transformers==4.4.0
!pip install torchaudio
!pip install librosa
!pip install jiwer

In [5]:
import os
import pickle
import random
import librosa
import numpy as np
import pandas as pd
import IPython.display as ipd
from datasets import ClassLabel, load_dataset, load_metric
from sklearn.model_selection import train_test_split
from IPython.display import Audio, Javascript, display, HTML, Image
from transformers import Wav2Vec2CTCTokenizer, Wav2Vec2ForCTC, Wav2Vec2Processor, TrainingArguments, Wav2Vec2FeatureExtractor

In [ ]:
train = load_dataset('csv', data_files='dvoice-v1.0/transcriptions/train.csv')
test = load_dataset('csv', data_files='dvoice-v1.0/transcriptions/test.csv')

dvoice_train = train['train']
dvoice_test = test['train']

# 2. Préparation des données

## 2.1. Creation du Tokenizer

In [ ]:
def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset)
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

show_random_elements(dvoice_train.remove_columns(["path"]), num_examples=5)

In [ ]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�]'

def remove_special_characters(batch):
    batch["transcript"] = re.sub(chars_to_ignore_regex, '', batch["transcript"]).lower() + " "
    return batch

In [ ]:
dvoice_train = dvoice_train.map(remove_special_characters)
dvoice_test = dvoice_test.map(remove_special_characters)

In [ ]:
show_random_elements(dvoice_train.remove_columns(["path"]))

In [ ]:
def extract_all_chars(batch):
  all_text = " ".join(batch["transcript"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [ ]:
vocab_train = dvoice_train.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=dvoice_train.column_names)
vocab_test = dvoice_test.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=dvoice_test.column_names)

In [ ]:
vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0]))

In [ ]:
vocab_dict = {v: k for k, v in enumerate(vocab_list)}
# vocab_dict

In [ ]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [ ]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

In [ ]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [ ]:
tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

## 2.2. Extraction des features

In [ ]:
feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)
processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

## 2.3. Prétraitement

In [ ]:
import torchaudio
def speech_file_to_array_fn(batch):
    speech_array, sampling_rate = torchaudio.load("dvoice_augmented/audios/" + batch["path"])
    batch["speech"] = speech_array[0].numpy()
    batch["sampling_rate"] = sampling_rate
    batch["target_text"] = batch["transcript"]
    return batch

In [ ]:
dvoice_train = dvoice_train.map(speech_file_to_array_fn, remove_columns=dvoice_train.column_names)
dvoice_test = dvoice_test.map(speech_file_to_array_fn, remove_columns=dvoice_test.column_names)

In [ ]:
def resample(batch):
    batch["speech"] = librosa.resample(np.asarray(batch["speech"]), batch['sampling_rate'], 16_000)
    batch["sampling_rate"] = 16_000
    return batch

In [ ]:
dvoice_train = dvoice_train.map(resample, num_proc=4)
dvoice_test = dvoice_test.map(resample, num_proc=4)

In [ ]:
rand_int = random.randint(0, len(dvoice_train)-1)
print("Target text:", dvoice_train[rand_int]["target_text"])
print("Input array shape:", np.asarray(dvoice_train[rand_int]["speech"]).shape)
print("Sampling rate:", dvoice_train[rand_int]["sampling_rate"])

ipd.Audio(data=np.asarray(dvoice_train[rand_int]["speech"]), autoplay=True, rate=16000)

In [ ]:
def prepare_dataset(batch):
    assert (
        len(set(batch["sampling_rate"])) == 1
    ), f"Make sure all inputs have the same sampling rate of {processor.feature_extractor.sampling_rate}."

    batch["input_values"] = processor(batch["speech"], sampling_rate=batch["sampling_rate"][0]).input_values
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["target_text"]).input_ids
    return batch

In [ ]:
dvoice_train = dvoice_train.map(prepare_dataset, remove_columns=dvoice_train.column_names, batch_size=8, num_proc=4, batched=True)
dvoice_test = dvoice_test.map(prepare_dataset, remove_columns=dvoice_test.column_names, batch_size=8, num_proc=4, batched=True)

# 3. Entraînement

## 3.1. Configurations

In [ ]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [ ]:
wer_metric = load_metric("wer", cache_dir="./")
# cer_metric = load_metric("cer", cache_dir="./")

In [ ]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)
#     cer = cer_metric.compute(predictions=pred_str, references=label_str)
    
    return {"wer": wer}
#     return {"cer" : cer}

In [ ]:
model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-large-xlsr-53", 
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
    gradient_checkpointing=True, 
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer)
)

In [ ]:
model.freeze_feature_extractor()

In [ ]:
training_args = TrainingArguments(
  output_dir="./dvoice-v1.0",
  group_by_length=True,
  per_device_train_batch_size=16,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=30,
  fp16=True,
  save_steps=400,
  eval_steps=400,
  logging_steps=400,
  learning_rate=3e-4,
  warmup_steps=500,
  save_total_limit=2,
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=dvoice_train,
    eval_dataset=dvoice_test,
    tokenizer=processor.feature_extractor,
)

## 3.2. Entraînement du modèle

In [ ]:
trainer.train()